In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_infro = tfds.load(name='mnist', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\marqu\tensorflow_datasets\mnist\incomplete.DC90ME_3.0.1\mnist-train.tfrecord*...:   0%|    …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\marqu\tensorflow_datasets\mnist\incomplete.DC90ME_3.0.1\mnist-test.tfrecord*...:   0%|     …

Dataset mnist downloaded and prepared to C:\Users\marqu\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_val = 0.1* mnist_infro.splits['train'].num_examples
num_val = tf.cast(num_val, tf.int64)

num_test = mnist_infro.splits['test'].num_examples
num_test = tf.cast(num_test, tf.int64)

def Scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_val_data = mnist_train.map(Scale)
test_data = mnist_test.map(Scale)

BUFFER_SIZE = 10000

shuffled_train_val_data = scaled_train_val_data.shuffle(BUFFER_SIZE)

val_data = shuffled_train_val_data.take(num_val)
train_data = shuffled_train_val_data.skip(num_val)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
val_data = val_data.batch(num_val)
test_data = test_data.batch(num_test)

validation_inputs, validation_targets = next(iter(val_data))

In [31]:
input_size = 784
output_size = 10
hidden_layer_size = 500

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

In [32]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) #one-hot encoding

In [33]:
NUM_EPCOHS = 5

model.fit(train_data, epochs=NUM_EPCOHS, validation_data=(validation_inputs,validation_targets), verbose=2)

Epoch 1/5
540/540 - 6s - 11ms/step - accuracy: 0.9296 - loss: 0.2332 - val_accuracy: 0.9667 - val_loss: 0.1163
Epoch 2/5
540/540 - 5s - 10ms/step - accuracy: 0.9700 - loss: 0.1004 - val_accuracy: 0.9715 - val_loss: 0.0967
Epoch 3/5
540/540 - 5s - 10ms/step - accuracy: 0.9775 - loss: 0.0770 - val_accuracy: 0.9777 - val_loss: 0.0740
Epoch 4/5
540/540 - 5s - 10ms/step - accuracy: 0.9836 - loss: 0.0544 - val_accuracy: 0.9772 - val_loss: 0.0819
Epoch 5/5
540/540 - 5s - 9ms/step - accuracy: 0.9859 - loss: 0.0475 - val_accuracy: 0.9853 - val_loss: 0.0525


In [34]:
test_loss, test_accuracy = model.evaluate(test_data)

print('test loss:', test_loss)
print('test accuracy:', test_accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.9739 - loss: 0.1007
test loss: 0.10067973285913467
test accuracy: 0.9739000201225281
